In [6]:
from huggingface_hub import notebook_login

notebook_login()

In [1]:
import os
import gc
import torch
import transformers
from transformers import (
    AutoModelForCausalLM, 
    AutoTokenizer, 
    TrainingArguments, 
    BitsAndBytesConfig
)
from datasets import load_dataset
from peft import (
    LoraConfig, 
    PeftModel, 
    get_peft_model, 
    prepare_model_for_kbit_training
)
from trl import DPOTrainer, setup_chat_format
import bitsandbytes as bnb

2024-07-16 09:33:36.703468: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-16 09:33:38.455112: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Load the model and tokenizer

Note that we also load in a reference model. This is for completeness. If we did not provide one, the DPOTrainer will automatically create one for us

In [30]:
base_model = "meta-llama/Meta-Llama-3-8B"
new_model = "DPOLlama-3-8B"

In [10]:
# The bits and bytes config we use for quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

In [11]:
tokenizer = AutoTokenizer.from_pretrained(base_model)
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)
# Reference model
ref_model = AutoModelForCausalLM.from_pretrained(
    base_model,
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [12]:
# LoRA configuration
peft_config = LoraConfig(
    r=16,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['k_proj', 'gate_proj', 'v_proj', 'up_proj', 'q_proj', 'o_proj', 'down_proj']
)

## Load and format dataset

Preference datasets are a little unique as I explained above. Let's take a look

In [13]:
# Load dataset
dataset = load_dataset("Intel/orca_dpo_pairs")['train']

Generating train split: 0 examples [00:00, ? examples/s]

In [14]:
# In this notebook we use a subset of 150 samples. In order to do a full tune, you can use around 1000 or the entire dataset
# The seed allows for reproducability. If you run this notebook exactly, you will use the same 150 samples as me
dataset = dataset.shuffle(seed=42).select(range(150))

In [15]:
dataset

Dataset({
    features: ['system', 'question', 'chosen', 'rejected'],
    num_rows: 150
})

In [16]:
dataset[10]

{'system': 'You are an AI assistant that follows instruction extremely well. Help as much as you can.',
 'question': 'What key details about al-walid i  can be extracted from the following bio?  Bio: al-walid ibn abd al-malik -lrb- -rrb- or al-walid i -lrb- 668 -- 23 february 715 -rrb- was an umayyad caliph who ruled from 705 until his death in 715 . his reign saw the greatest expansion of the caliphate , as successful campaigns were undertaken in transoxiana , sind , hispania and against the byzantines .\n',
 'chosen': '1. Name: Al-Walid ibn Abd al-Malik or Al-Walid I\n2. Birth year: 668\n3. Death date: 23 February 715\n4. Title: Umayyad Caliph\n5. Reign: Ruled from 705 to 715\n6. Key accomplishment: His reign saw the greatest expansion of the Caliphate\n7. Successful campaigns: Undertaken in Transoxiana, Sind, Hispania, and against the Byzantines',
 'rejected': " Sure, I can help you extract some key details about al-Walid ibn Abd al-Malik from the given bio. Here are some of the inf

In [17]:
dataset[19]['question']

'WWE star invites Conor McGregor to join WWE\nConor McGregor turned the UFC world upside down in less than 140 characters Tuesday, cryptically tweeting that he has made the decision to "retire young."\n\nI have decided to retire young.Thanks for the cheese. Catch ya\'s later.\n\nNo one really knows what McGregor means. He may have just announced his retirement at the age of 27, or perhaps McGregor is simply sharing that he doesn\'t plan to fight well into his 30s. As one of the biggest stars in sports, and at the height of his earning potential, it would seem crazy for McGregor to actually retire - but he would certainly have lucrative options outside of the octagon.\n\nMcGregor\'s Irish compatriot Becky Lynch suggested on Twitter that the UFC superstar should make the jump to professional wrestling and join WWE.\n\nMcGregor wouldn\'t be the first MMA star to transition to WWE. "The World\'s Most Dangerous Man" Ken Shamrock is the best example of a success story. Shamrock came to WWE i

In [18]:
dataset[19]['rejected']

" The article discusses the possibility of Conor McGregor, a UFC star, joining WWE. McGregor has announced his retirement from the UFC, and WWE star Becky Lynch has suggested that he should join WWE. The article explores the potential of McGregor joining WWE, including the possibility of him being a massive draw, but also notes that it would be challenging to integrate him into the WWE roster and storylines. The article also mentions other MMA stars who have transitioned to WWE, such as Ken Shamrock and Brock Lesnar. Additionally, the article suggests that McGregor's character and personality would be a major factor in any WWE storyline."

In [19]:
dataset[19]['chosen']

'This article is about the speculation surrounding UFC star Conor McGregor\'s potential retirement from the sport after his cryptic tweet hinting at "retiring young." The article discusses the suggestion made by McGregor\'s Irish compatriot Becky Lynch that he should transition from UFC to professional wrestling, specifically WWE. The possibility of McGregor joining WWE is analysed, drawing comparisons with other MMA stars who have made the switch, such as Ken Shamrock, Dan Severn, Alberto Del Rio, Shinsuke Nakamura, and Brock Lesnar.\n\nThe article addresses various factors that would need to be considered, such as McGregor\'s professional wrestling training, potential character challenges, and the scope of his involvement in WWE storylines. It concludes by contemplating the idea of McGregor taking part in occasional high-profile WWE pay-per-view events, similar to Brock Lesnar\'s approach, rather than becoming a regular main eventer.'

Now we format the dataset in order to follow the Llama3 format using the extremely helpful chat templates created by chujiezheng

In [20]:
!wget -L https://raw.githubusercontent.com/chujiezheng/chat_templates/main/chat_templates/llama-3-instruct.jinja

--2024-07-16 09:42:04--  https://raw.githubusercontent.com/chujiezheng/chat_templates/main/chat_templates/llama-3-instruct.jinja
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 598 [text/plain]
Saving to: ‘llama-3-instruct.jinja’

llama-3-instruct.ji 100%[===================>]     598  --.-KB/s    in 0s      

2024-07-16 09:42:05 (35.5 MB/s) - ‘llama-3-instruct.jinja’ saved [598/598]



In [21]:
chat_template = open('llama-3-instruct.jinja').read()
chat_template = chat_template.replace('    ', '').replace('\n', '')
tokenizer.chat_template = chat_template

In [22]:
def dataset_format(example):
    # Format system
    if len(example['system']) > 0:
        message = {"role": "system", "content": example['system']}
        system = tokenizer.apply_chat_template([message], tokenize=False)
    else:
        system = ""
    # Format instruction
    message = {"role": "user", "content": example['question']}
    prompt = tokenizer.apply_chat_template([message], tokenize=False, add_generation_prompt=True)
    # Format chosen answer
    chosen = example['chosen'] + "<|eot_id|>\n"
    # Format rejected answer
    rejected = example['rejected'] + "<|eot_id|>\n"
    return {
        "prompt": system + prompt,
        "chosen": chosen,
        "rejected": rejected,
    }

In [23]:
original_columns = dataset.column_names
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

dataset = dataset.map(
    dataset_format,
    remove_columns=original_columns,
    num_proc= os.cpu_count(),
)

Map (num_proc=32):   0%|          | 0/150 [00:00<?, ? examples/s]

In [24]:
# notice the specific llama3 tags like <|eot_id|> which show that the chat template formatting worked
dataset[19]

{'chosen': 'This article is about the speculation surrounding UFC star Conor McGregor\'s potential retirement from the sport after his cryptic tweet hinting at "retiring young." The article discusses the suggestion made by McGregor\'s Irish compatriot Becky Lynch that he should transition from UFC to professional wrestling, specifically WWE. The possibility of McGregor joining WWE is analysed, drawing comparisons with other MMA stars who have made the switch, such as Ken Shamrock, Dan Severn, Alberto Del Rio, Shinsuke Nakamura, and Brock Lesnar.\n\nThe article addresses various factors that would need to be considered, such as McGregor\'s professional wrestling training, potential character challenges, and the scope of his involvement in WWE storylines. It concludes by contemplating the idea of McGregor taking part in occasional high-profile WWE pay-per-view events, similar to Brock Lesnar\'s approach, rather than becoming a regular main eventer.<|eot_id|>\n',
 'rejected': " The articl

## Create the DPO trainer

In [ ]:
import wandb

wandb.login()

In [31]:
training_args = TrainingArguments(
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    learning_rate=5e-5,
    lr_scheduler_type="cosine",
    max_steps=50, #tweak this to change # of steps in the training run
    save_strategy="no",
    logging_steps=1,
    output_dir=new_model,
    optim="paged_adamw_32bit",
    warmup_steps=10,
    bf16=True,
    report_to="wandb",
)

In [32]:
dpo_trainer = DPOTrainer(
    model,
    ref_model,
    args=training_args,
    train_dataset=dataset,
    tokenizer=tokenizer,
    peft_config=peft_config,
    beta=0.1,
    max_prompt_length=512,
    max_length=1024, 
    force_use_ref_model=True
)

Map:   0%|          | 0/150 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [33]:
# Fine-tune model with DPO
dpo_trainer.train()

Step,Training Loss
1,0.698500
2,0.700300
3,0.676000
4,0.668000
5,0.649400
6,0.613400
7,0.589000
8,0.441000
9,0.413500
10,0.422300


TrainOutput(global_step=50, training_loss=0.19132064570963847, metrics={'train_runtime': 321.0201, 'train_samples_per_second': 1.246, 'train_steps_per_second': 0.156, 'total_flos': 0.0, 'train_loss': 0.19132064570963847, 'epoch': 2.6666666666666665})

## Save and test model

In [35]:
dpo_trainer.model.save_pretrained("final_ckpt")

/home/test/anaconda3/envs/gvenv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [36]:
tokenizer.save_pretrained("final_ckpt")

('final_ckpt/tokenizer_config.json',
 'final_ckpt/special_tokens_map.json',
 'final_ckpt/tokenizer.json')

In [37]:
# Flush memory
del dpo_trainer, model, ref_model
gc.collect()
torch.cuda.empty_cache()

In [38]:
# Reload model in FP16 (instead of NF4)
base_model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Meta-Llama-3-8B-Instruct",
    return_dict=True,
    torch_dtype=torch.float16,
)

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [39]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct")
tokenizer.chat_template = chat_template

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [40]:
# Merge base model with the adapter
model = PeftModel.from_pretrained(base_model, "final_ckpt")
model = model.merge_and_unload()

# Save model and tokenizer
model.save_pretrained(new_model)
tokenizer.save_pretrained(new_model)

('DPOLlama-3-8B/tokenizer_config.json',
 'DPOLlama-3-8B/special_tokens_map.json',
 'DPOLlama-3-8B/tokenizer.json')

In [41]:
# Create pipeline
pipeline = transformers.pipeline(
    "text-generation",
    model=new_model,
    tokenizer=tokenizer
)

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [42]:
# Format prompt
message = [
    {"role": "system", "content": "You are a helpful assistant chatbot that provides concise answers."},
    {"role": "user", "content": "What are GPUs and why would I use them for machine learning tasks?"}
]
tokenizer = AutoTokenizer.from_pretrained(new_model)
prompt = tokenizer.apply_chat_template(message, add_generation_prompt=True, tokenize=False)

# Generate text
sequences = pipeline(
    prompt,
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
    num_return_sequences=1,
    max_length=200,
)
print(sequences[0]['generated_text'])

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful assistant chatbot that provides concise answers.<|eot_id|><|start_header_id|>user<|end_header_id|>

What are GPUs and why would I use them for machine learning tasks?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

GPUs (Graphics Processing Units) are specialized computer chips designed for high-performance computing, particularly for graphics rendering. They have evolved to also excel in accelerating general-purpose computing, including machine learning tasks.

GPUs offer several advantages for machine learning:

1. **Parallel processing**: GPUs have thousands of cores, allowing them to perform many calculations simultaneously. This parallel processing capability enables GPUs to handle large datasets and complex computations more efficiently than CPUs (Central Processing Units).
2. **Massive memory bandwidth**: GPUs have a high memory bandwidth, which enables fast data transfer between the GPU and s